In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import os

import modules.logger_tool as logger
os.environ['LOG_NAME'] = 'Calendar'
os.environ['LOG_DIR'] = 'logs'
os.environ['LOG_LEVEL'] = 'INFO'

logging = logger.get_logger(os.environ['LOG_NAME'], log_level=os.environ['LOG_LEVEL'], log_path=os.environ['LOG_DIR'], log_file=os.environ['LOG_NAME'])


In [2]:
import modules.database.schemas.calendar_neo as calendar_neo
import modules.database.schemas.relationships.calendar_rels as cal_rels
import modules.database.tools.neontology_tools as neon
import modules.database.tools.neo4j_driver_tools as driver_tools

import requests
from datetime import timedelta, datetime, date


In [4]:
db_name = os.environ['LOG_NAME']


In [5]:
url = f'http://{os.environ["BACKEND_URL"]}:{os.environ("BACKEND_PORT")}/database/admin/stop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = f'http://{os.environ["BACKEND_URL"]}:{os.environ("BACKEND_PORT")}/database/admin/drop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = f'http://{os.environ["BACKEND_URL"]}:{os.environ("BACKEND_PORT")}/database/admin/create-database'
params = {'db_name': db_name}
response = requests.post(url, params=params)
logging.info(response.text)

2024-07-06 14:13:37,694 INFO      : 3522179693 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAqZA="]}
2024-07-06 14:13:39,875 INFO      : 3522179693 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAqpA="]}


In [7]:
# Initialize driver
driver = driver_tools.get_driver(database=db_name)

# Initialize connection
neon.init_neo4j_connection()


2024-07-06 14:13:41,987 INFO      : neo4j_driver_tools > get_driver >>> Connection successful


In [9]:
# Define the calendar range
start_date = date(2019, 1, 1)
end_date = date(2024, 12, 31)


In [11]:
def create_calendar(start_date, end_date, attach=False):
    # Dictionary to track created nodes to avoid duplicates
    created_years = {}
    created_months = {}
    created_weeks = {}
    created_days = {}

    # Variables to store the last node of each type
    last_year_node = None
    last_month_node = None
    last_week_node = None
    last_day_node = None

    # Dictionary to store the nodes and relationships
    calendar_nodes = []
    year_nodes = []
    month_nodes = []
    week_nodes = []
    day_nodes = []
    nodes = {
        'calendar_node': calendar_nodes,
        'year_nodes': year_nodes,
        'month_nodes': month_nodes,
        'week_nodes': week_nodes,
        'day_nodes': day_nodes
    }
    
    current_date = start_date

    if attach:
        calendar_node = calendar_neo.CalendarNode(
            calendar_id=f"calendar_{db_name}",
            calendar_name=db_name
        )
        neon.create_or_merge_neontology_node(calendar_node, database=db_name, operation='merge')
        nodes['calendar_nodes'].append(calendar_node)

    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        day = current_date.day
        iso_year, iso_week, iso_weekday = current_date.isocalendar()

        # Ensure year node is created for the Gregorian calendar year
        if year not in created_years:
            year_node = calendar_neo.CalendarYearNode(
                calendar_year_id=f"calendar_year_{year}",
                year=year,
                iso_year=str(year),
                localised_year_name=f"Year {year}"
            )
            neon.create_or_merge_neontology_node(year_node, database=db_name, operation='merge')
            nodes['year_nodes'].append(year_node)
            created_years[year] = year_node
            
            if attach:
                neon.create_or_merge_neontology_relationship(
                    cal_rels.CalendarIncludesYear(source=calendar_node, target=year_node),
                    database=db_name,
                    operation='merge'
                )

            if last_year_node:
                neon.create_or_merge_neontology_relationship(
                    cal_rels.YearFollowsYear(source=last_year_node, target=year_node),
                    database=db_name,
                    operation='merge'
                )
            last_year_node = year_node

        # Ensure month node is created
        month_key = f"{year}-{month}"
        if month_key not in created_months:
            month_node = calendar_neo.CalendarMonthNode(
                calendar_month_id=f"calendar_month_{year}_{month}",
                month=month,
                month_name=datetime(year, month, 1).strftime('%B'),
                iso_month=f"{year}-{month:02}",
                localised_month_name=f"{datetime(year, month, 1).strftime('%B')} {year}"
            )
            neon.create_or_merge_neontology_node(month_node, database=db_name, operation='merge')
            nodes['month_nodes'].append(month_node)
            created_months[month_key] = month_node

            # Check for the end of year transition for months
            if last_month_node:
                if month == 1 and last_month_node.month == 12 and int(last_month_node.calendar_month_id.split('_')[1]) == year - 1:
                    neon.create_or_merge_neontology_relationship(
                        cal_rels.MonthFollowsMonth(source=last_month_node, target=month_node),
                        database=db_name,
                        operation='merge'
                    )
                elif month == last_month_node.month + 1:
                    neon.create_or_merge_neontology_relationship(
                        cal_rels.MonthFollowsMonth(source=last_month_node, target=month_node),
                        database=db_name,
                        operation='merge'
                    )

            last_month_node = month_node

            neon.create_or_merge_neontology_relationship(
                cal_rels.YearIncludesMonth(source=year_node, target=month_node),
                database=db_name,
                operation='merge'
            )

        # Week node management
        week_key = f"{iso_year}-W{iso_week}"
        if week_key not in created_weeks:
            week_node = calendar_neo.CalendarWeekNode(
                calendar_week_id=f"calendar_week_{iso_year}_{iso_week}",
                week_number=iso_week,
                iso_week=f"{iso_year}-W{iso_week:02}",
                localised_week_name=f"Week {iso_week}, {iso_year}"
            )
            neon.create_or_merge_neontology_node(week_node, database=db_name, operation='merge')
            nodes['week_nodes'].append(week_node)
            created_weeks[week_key] = week_node

            if last_week_node and ((last_week_node.iso_week.split('-')[0] == str(iso_year) and last_week_node.week_number == iso_week - 1) or
                                (last_week_node.iso_week.split('-')[0] != str(iso_year) and last_week_node.week_number == 52 and iso_week == 1)):
                neon.create_or_merge_neontology_relationship(
                    cal_rels.WeekFollowsWeek(source=last_week_node, target=week_node),
                    database=db_name,
                    operation='merge'
                )
            last_week_node = week_node

            neon.create_or_merge_neontology_relationship(
                cal_rels.YearIncludesWeek(source=year_node, target=week_node),
                database=db_name,
                operation='merge'
            )

        # Day node management
        day_key = f"{year}-{month}-{day}"
        day_node = calendar_neo.CalendarDayNode(
            calendar_day_id=f"calendar_day_{year}_{month}_{day}",
            date=current_date,
            day_of_week=current_date.strftime('%A'),
            iso_day=f"{year}-{month:02}-{day:02}",
            localised_day_name=f"{current_date.strftime('%A')}, {current_date.strftime('%B')} {day}, {year}",
            is_weekend=current_date.weekday() > 4
        )
        neon.create_or_merge_neontology_node(day_node, database=db_name, operation='merge')
        nodes['day_nodes'].append(day_node)
        created_days[day_key] = day_node

        if last_day_node:
            neon.create_or_merge_neontology_relationship(
                cal_rels.DayFollowsDay(source=last_day_node, target=day_node),
                database=db_name,
                operation='merge'
            )
        last_day_node = day_node

        neon.create_or_merge_neontology_relationship(
            cal_rels.MonthIncludesDay(source=month_node, target=day_node),
            database=db_name,
            operation='merge'
        )

        neon.create_or_merge_neontology_relationship(
            cal_rels.WeekIncludesDay(source=week_node, target=day_node),
            database=db_name,
            operation='merge'
        )

        current_date += timedelta(days=1)
        
        return nodes

In [ ]:
calendar_nodes = create_calendar(start_date, end_date, attach=True)
